In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import pickle
import numpy as np
import pandas as pd

In [3]:
examples = pd.read_csv('./computational_examples.csv')

In [4]:
len(examples)

3043

# create the master set of ratings to use in training

In [12]:
computational_examples = []
for hash in examples['hash'].unique():
    hash_examples = examples[examples['hash'] == hash]
    video_examples = hash_examples[hash_examples['condition'] == 'video']
    audio_examples = hash_examples[hash_examples['condition'] == 'audio']
    av_examples = hash_examples[hash_examples['condition'] == 'av']

    num_examples = min(len(video_examples), len(audio_examples), len(av_examples))

    if num_examples != 2:
        print(f'{hash} -> {num_examples}')

    # print(num_examples)
    computational_examples.append(video_examples.iloc[:num_examples, :])
    computational_examples.append(audio_examples.iloc[:num_examples, :])
    computational_examples.append(av_examples.iloc[:num_examples, :])

bb6337eea970487ce9cd4ff26ea78c7acc6d5d1a355b7aa50029a3229f115b21 -> 0
68d229cf19eec82f37580265ea93892117dd5b559b04d22489da3593315f18e7 -> 0
ced6e78fe7940c10fbc9d7c385273e68459ca399ccb668c8123cf5a66fa99819 -> 0
64a92aea9395ace7ac9d60eab34911e419fc66610cd76f1e29df4b4fd16f230f -> 0
b438e94f7fdcc80ea7927e320946c7b407b7c850fe04baa60a07a5df7d92a711 -> 0
85aac70ec91eb3be1b313b33e0b7828394bbe4e4edc6a956d1e7061dfc8b250e -> 0
952a739cb6f9de3aede44e80cc7ef0b7d5de5f5840bcc9873918575248eec210 -> 1
2813032c9ff3b525c802dfe1a6d197cca3003c358389405393e2d6de90de2358 -> 1
17c03da0fbb85a73f02bfbb1b699c4effaf9f23ab4ed97fd2e919f93925f2218 -> 1
12b5c620b18a8354b1c2368935c52d9507c1144e7af2511226bbb1447398ee0c -> 1
ec01596ea5013f5178d2d4f1241660e09e4cbd78c32b68de15cab3b0ee496a7c -> 1
942a4bffa10339350bdca866df2e8d33e16c692be35904018e9c391cc018d953 -> 1
c489fd878e5060e9757d2dbe04ecade9166b500e7b6494e77586fd51c19a0370 -> 1
1ed802be09b068291ab527e2903f91430930efc826007e718ef093193f1e2396 -> 1
d61adf3841c03271453a

In [8]:
computational_examples_df = pd.concat(computational_examples)

In [9]:
len(computational_examples_df)

2016

In [13]:
computational_examples_df.to_csv('./computational_examples.csv')

# Binary labels

In [14]:
examples.head()

,Unnamed: 0,person,cam,hit_id,condition,calibration,hash,ini_time,end_time,_ini_time,...,gt_offset,gt_laughter,is_laughter,confidence,intensity,attempt,pressed_key,onset,offset,rating_hash
0,0,25,1,9c45e4f0c5442e796eb93e73e94dc6c2dfca7b9c4c54ff...,video,False,1170917790b51bc5a8dacacc4d8ed8c410b7ea6bb7ea4b...,7360.290,7361.54,7358.022087,...,4.420238,True,True,7,4,0,True,3.336670,6.639973,7af591213b827db95c12c56e76e0b1fe518f2088d11aad...
1,1,35,3,9c45e4f0c5442e796eb93e73e94dc6c2dfca7b9c4c54ff...,audio,False,11bc9d8aca57ab2aef4c5305b080fa49c08665d9e94190...,2216.020,2216.54,2213.600368,...,3.928860,True,True,7,2,0,True,2.943220,3.583220,9681c8a0e3fe8bd905960c03539881a978f8afdba94cc4...
2,2,1,4,9c45e4f0c5442e796eb93e73e94dc6c2dfca7b9c4c54ff...,video,False,c1d181e74dbdbce1e51d7d0bfd6e036913896dd1f22856...,3346.300,3347.70,3344.741847,...,3.255518,True,True,2,2,0,True,2.736069,3.203203,07d4ee10402ea059d0a3791fd35fbaab20149aeb6ffb99...
3,3,35,3,9c45e4f0c5442e796eb93e73e94dc6c2dfca7b9c4c54ff...,audio,False,b3710a5f8dc552872b8f7ff9be406711345f4892e836f2...,1139.890,1143.13,1137.518260,...,6.403030,True,True,7,6,0,True,3.063220,5.503220,bad1548826508a93cedf5e2e442e856d00ab038e06b870...
4,4,21,1,9c45e4f0c5442e796eb93e73e94dc6c2dfca7b9c4c54ff...,av,False,cbc382abda5165dd26a7ca9e05e7ed3000933489864cb9...,4811.917,4813.35,4808.965641,...,4.150842,True,True,7,2,0,True,3.626824,4.427624,d8b65d40196ef68ccce5d07e04fc6004734d6900059bc6...


In [14]:
video_labels = {row['rating_hash']: row['pressed_key'] for _, row in examples[examples['condition'] == 'video'].iterrows()}
audio_labels = {row['rating_hash']: row['pressed_key'] for _, row in examples[examples['condition'] == 'audio'].iterrows()}
av_labels = {row['rating_hash']: row['pressed_key'] for _, row in examples[examples['condition'] == 'av'].iterrows()}

In [16]:
print((len(video_labels), len(audio_labels), len(av_labels)))

(1020, 1020, 1003)


In [20]:
pickle.dump(video_labels, open('labels/video_pk.pkl', 'wb'))
pickle.dump(audio_labels, open('labels/audio_pk.pkl', 'wb'))
pickle.dump(av_labels, open('labels/av_pk.pkl', 'wb'))